# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Heart-Failure-Prediction-hyperParameterTuning'

experiment=Experiment(ws, experiment_name)
  #Create an enviroment
#env=Environment(workspace=ws,name='MyEnvironment')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RYCTLQL78 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
#Create compute cluster
# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)
    print('cpu-cluster has been created')

Creating a new compute target ...
cpu-cluster has been created


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": choice(0.6,1,1.5),
    "--max_iter": choice(10,20,30,40)})
#compute_target=ws.compute_targets['cpu-cluster']
#TODO: Create your estimator and hyperdrive config
scr = SKLearn(source_directory='./',
             entry_script='train.py',  
             compute_target='cpu-cluster'
             )

hyperdrive_run_config = HyperDriveConfig(estimator=scr,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4 
                                    
                                    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
#TODO: Submit your experiment
hyperdrive_run=experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
# Get your best run and save the model from that run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run ID: ', best_run.id)
print('Accuracy: ', best_run_metrics['Accuracy'])
print('Hyperparameter Values: ',best_run.get_details()['runDefinition']['arguments'])
print('Best Run Files:\n ',best_run.get_file_names())

Best Run ID:  HD_6740bb29-c510-4046-985a-6ed3c8066ad2_11
Accuracy:  0.88
Hyperparameter Values:  ['--C', '1.5', '--max_iter', '10']
Best Run Files:
  ['azureml-logs/55_azureml-execution-tvmps_be71f3973e1168a41d868414a90191d138619dfefa1d61a0e37fc321cc9b6222_d.txt', 'azureml-logs/65_job_prep-tvmps_be71f3973e1168a41d868414a90191d138619dfefa1d61a0e37fc321cc9b6222_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_be71f3973e1168a41d868414a90191d138619dfefa1d61a0e37fc321cc9b6222_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/103_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_bdc454cc-914b-4f7e-afc9-fb562d946859.jsonl', 'logs/azureml/dataprep/python_span_l_bdc454cc-914b-4f7e-afc9-fb562d946859.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.pkl']


In [9]:
#TODO: Save the best model
model =best_run.register_model(model_name = 'hyperdrive_best_model', model_path='./outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service